In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor

from sklearn import metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score

from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import metrics
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from skompiler import skompile

In [2]:
veri_seti = pd.read_excel('HW_Data_Set.xlsx')
df = veri_seti.copy()

In [3]:
# ? leri için columnsları int çevirip, eksik gözlem olarak aldım.
df['ind_420'] = pd.to_numeric(df['ind_420'], errors='coerce')
df['ind_422'] = pd.to_numeric(df['ind_422'], errors='coerce')

df["ind_422"].fillna(df["ind_422"].median(), inplace=True)
df["ind_420"].fillna(df["ind_420"].median(), inplace=True)

In [4]:
df_dummies = pd.get_dummies(df)

In [5]:
X = df_dummies.drop(["20_target","50_target","90_target"], axis = 1)
y = df_dummies["50_target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, shuffle =  False)

# SVR

In [6]:
svr = SVR()

In [7]:
svr_model = svr.fit(X_train, y_train)
y_pred = svr_model.predict(X_test)

mse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("mse: {} \nr2: {}".format(mse,r2))

mse: 41.35197685503661 
r2: -0.4486137160364716


In [8]:
svr_model

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [9]:
svr_params = {"kernel": ["linear", 'poly', 'rbf'],
             "C": [2,3,6,8]}

In [10]:
svr_model_cv = GridSearchCV(svr, svr_params, cv=5, n_jobs= -1, verbose= 2)

In [ ]:
svr_model_cv.fit(X_train, y_train)

In [10]:
svr_model_tuned = SVR(C= 8, kernel = 'rbf').fit(X_train, y_train)
y_pred_tuned = svr_model_tuned.predict(X_test)



mse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
r2_tuned = r2_score(y_test, y_pred_tuned)

print("mse: {} \nr2: {}\n*********************************\nmse tuned: {} \nr2 tuned: {}".format(mse,r2, mse_tuned, r2_tuned))

mse: 41.35197685503661 
r2: -0.4486137160364716
*********************************
mse tuned: 42.33295054820127 
r2 tuned: -0.5181585133202093


# Gradient Boosting

In [6]:
gb = GradientBoostingRegressor()

In [7]:
gb_model = gb.fit(X_train, y_train)
y_pred = gb_model.predict(X_test)

mse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("mse: {} \nr2: {}".format(mse,r2))

mse: 39.14414516491361 
r2: -0.2980567127844096


In [8]:
gb

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [9]:
gb_params = {"learning_rate": [0.1, 0.2, 0.3],
            'n_estimators': [100, 200, 500],
            'subsample': [0.6, 0.8, 1.0],
            'max_depth': [3, 5, 15]}
gb_model_cv = GridSearchCV(gb, gb_params, cv=5, n_jobs= -1, verbose= 2)
gb_model_cv.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 46.1min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 52.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_n...None,
                       

In [10]:
gb_model_cv.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}

In [12]:
gb_model_tuned = GradientBoostingRegressor(learning_rate= 0.1, max_depth=3, n_estimators= 100, subsample= 0.8).fit(X_train, y_train)
y_pred_tuned = gb_model_tuned.predict(X_test)



mse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
r2_tuned = r2_score(y_test, y_pred_tuned)

print("mse: {} \nr2: {}\n*********************************\nmse tuned: {} \nr2 tuned: {}".format(mse,r2, mse_tuned, r2_tuned))

mse: 39.14414516491361 
r2: -0.2980567127844096
*********************************
mse tuned: 38.56160553693602 
r2 tuned: -0.259709069808294


# KNN REG

In [48]:
from sklearn.neighbors import KNeighborsRegressor

In [53]:
knnr = KNeighborsRegressor(n_neighbors=2)
knnr_model = knnr.fit(X_train, y_train)
y_pred = knnr_model.predict(X_test)
mse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("mse: {} \nr2: {}".format(mse,r2))

mse: 45.1239150993855 
r2: -0.7249383890054846


In [58]:
knnr_model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [59]:
knn_params = {"n_neighbors": np.arange(2,20)}

In [60]:
knnr_model_cv = GridSearchCV(knnr, knn_params, cv=5, n_jobs= -1, verbose= 2)
knnr_model_cv.fit(X_train, y_train)
knnr_model_cv.best_params_

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   19.9s finished


{'n_neighbors': 19}

In [64]:
knnr_tuned = KNeighborsRegressor(n_neighbors= 19)
knnr_model_tuned = knnr_tuned.fit(X_train, y_train)
y_pred_tuned = knnr_model_tuned.predict(X_test)

mse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
r2_tuned = r2_score(y_test, y_pred_tuned)

print("mse: {} \nr2: {}\n*********************************\nmse tuned: {} \nr2 tuned: {}".format(mse,r2, mse_tuned, r2_tuned))

mse: 45.1239150993855 
r2: -0.7249383890054846
*********************************
mse tuned: 37.30159736899904 
r2 tuned: -0.17873153048656842
